In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
import keras
from keras.datasets import fashion_mnist

In [3]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import math
import nibabel as nib
from nilearn import image
from scipy import ndimage
import glob
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [4]:
from sklearn.model_selection import train_test_split

In [48]:
from tensorflow.keras.preprocessing.image import load_img
import itertools
import shutil

In [8]:
def resize_data(data):
    initial_size_x = data.shape[0]
    initial_size_y = data.shape[1]
    initial_size_z = data.shape[2]

    new_size_x = 128
    new_size_y = 128
    new_size_z = 128

    delta_x = initial_size_x / new_size_x
    delta_y = initial_size_y / new_size_y
    delta_z = initial_size_z / new_size_z

    new_data = np.zeros((new_size_x, new_size_y, new_size_z))

    for x, y, z in itertools.product(range(new_size_x),
                                     range(new_size_y),
                                     range(new_size_z)):
        new_data[x][y][z] = data[int(x * delta_x)][int(y * delta_y)][int(z * delta_z)]

    return new_data

In [9]:
def preprocess_anatomical(img_data_array):
    brain_volume = []
    highest_dim = max(img_data_array.shape)
    modify_channel_start = int(highest_dim/4)
    modify_channel_end = int(min(img_data_array.shape))
    stop_point = modify_channel_start + modify_channel_end

    for i in range(modify_channel_start):
        temp = np.zeros((highest_dim, highest_dim))
        brain_volume.append(temp)

    for i in range(img_data_array.shape[2]):
        img_slice_array_ana = img_data_array[:,:,i]
    #     img_slice_array_ana  = ndimage.rotate(img_slice_array_ana, 90)
        brain_volume.append(img_slice_array_ana)

    for i in range(modify_channel_start):
        temp = np.zeros((highest_dim, highest_dim))
        brain_volume.append(temp)
        
    brain_volume = np.array(brain_volume)
#     brain_volume = brain_volume[40:214, 40:214, 40:214] 
    brain_volume = resize_data(brain_volume)
    return brain_volume

In [10]:
def preprocess_ground_truth(img_data_array_gt):
    brain_volume_gt = []
    highest_dim = max(img_data_array_gt.shape)
    modify_channel_start = int(highest_dim/4)
    modify_channel_end = int(min(img_data_array_gt.shape))
    stop_point = modify_channel_start + modify_channel_end

    for i in range(modify_channel_start):
        temp = np.zeros((highest_dim, highest_dim))
        brain_volume_gt.append(temp)

    for i in range(img_data_array_gt.shape[1]):
        img_slice_array_ana = img_data_array_gt[:,i,:]
    #     img_slice_array_ana  = ndimage.rotate(img_slice_array_ana, 90)
        brain_volume_gt.append(img_slice_array_ana)

    for i in range(modify_channel_start):
        temp = np.zeros((highest_dim, highest_dim))
        brain_volume_gt.append(temp)
        
    brain_volume_gt = np.array(brain_volume_gt)
    brain_volume_gt = brain_volume_gt[40:214,40:214, 40:214]
    brain_volume_gt = resize_data(brain_volume_gt)
    
    return  brain_volume_gt

In [24]:
# For anatomical slices
file_path = '......path'
files_mri  =  os.listdir(file_path)
files_mri  = [file for file in files_mri if  file.endswith('MR1')]
files_mri = sorted(files_mri)
paths_ana  = [file_path + '/' + file for file in files_mri]

In [26]:
for i in range(len(paths_ana )):
    part1 = paths_ana[i]
    part2 = '/PROCESSED/MPRAGE/T88_111/'
    path_given = [part1 + '/'+ part2]
    # MRI images
    different_mri_files =  os.listdir(path_given[0])
    files_to_mri  = [file for file in different_mri_files if  file.endswith('111_t88_gfc.img')]
    files_to_mri = sorted(files_to_mri)
    paths_ana_to_mri  = [path_given[0] + '/' + file for file in files_to_mri]
    
    img_anatomical = nib.load(paths_ana_to_mri[0])
    retreve_volume = image.index_img(img_anatomical, 0)
    # Get data from nibabel image object 
    img_data = retreve_volume.get_data()
    img_data_array = np.asarray(img_data)
    
    _, file_name_mri = os.path.split(paths_ana_to_mri[0])
    file_name_mri = file_name_mri[:file_name_mri.rfind(".")]
    pre_anatomical = resize_data(img_data_array)
    pre_anatomical = scaler.fit_transform(pre_anatomical.reshape(-1, pre_anatomical.shape[-1])).reshape(pre_anatomical.shape)
    pre_anatomical = np.stack([pre_anatomical], axis=3)
    np.save('mri_images/'+str(file_name_mri)+'.npy', pre_anatomical)

    
    # Ground truth images
    different_gt_files =  os.listdir(path_given[0])
    files_to_gt  = [file for file in different_gt_files if  file.endswith('masked_gfc.img')]
    files_to_gt = sorted(files_to_gt)
    paths_ana_to_gt = [path_given[0] + '/' + file for file in files_to_gt]
    
    img_gt = nib.load(paths_ana_to_gt[0])
    retreve_volume_gt = image.index_img(img_gt, 0)
    # Get data from nibabel image object 
    img_data_gt = retreve_volume_gt.get_data()
    img_data_array_gt = np.asarray(img_data_gt)
    img_data_test_binary = img_data_array_gt > 0
    
    _, file_name_mask = os.path.split(paths_ana_to_gt[0])
    file_name_mask = file_name_mask[:file_name_mask.rfind(".")]
    temp_mask = resize_data(img_data_test_binary)
    pre_gt = to_categorical(temp_mask, num_classes = 2)
    np.save('ground_truth_images/'+str(file_name_mask)+'.npy', pre_gt)